# Amazon Scraper
## ADS-509 Final
### Ivan Chavez, Uyen Pham, Lane Whitmore

In [109]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import random
import re
import itertools
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service
import collections


In [3]:
# top 50 urls for movies, books and video games
m_url = 'https://www.amazon.com/Best-Sellers-Movies-TV-Prime-Video/zgbs/movies-tv/2958933011/ref=zg_bs_nav_movies-tv_1'
b_url = 'https://www.amazon.com/gp/bestsellers/books/ref=s9_acss_bw_cg_bsmpill_1c1_w?ref=bsm_nav_pill_print&pf_rd_m=ATVPDKIKX0DER&pf_rd_s=merchandised-search-1&pf_rd_r=56RBBA17NCWBRFTBZVMP&pf_rd_t=101&pf_rd_p=ef8cebb8-ad4b-453c-8030-a931d3822444&pf_rd_i=16857165011'
vg_url = 'https://www.amazon.com/Best-Sellers-Video-Games-Xbox-Series-X-S-Games/zgbs/videogames/20972814011/ref=zg_bs_nav_videogames_2_20972798011'

In [13]:
# first the top 50 urls need to be scraped to grab the url for each product's description and review page
def scrape(url):
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    driver.get(url)

    time.sleep(8)
# refresh to get past amazon block
    driver.refresh()

# scroll page to find all 50
    driver.execute_script("window.scrollTo(0, 10000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 14000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 15000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 16000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 17000)") 


# lists to save the data to
    hrefs = []
    r_hrefs = []
    titles = []

    # grabbing the title of each product
    title = driver.find_elements_by_css_selector("div[class^='_cDEzb_p13n-sc-css-line-clamp-']")
    for t in title:
        if int(t.get_attribute('class').split('-')[-1].split('_')[0]) < 2:
            continue
        else:
            titles.append(t.text)
    
    # getting all elements that contain product href
    m_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@role='link']")
    
    # all elements containing 
    mr_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@title]")

    # loop through the elements to extract the href for both 
    for i in m_list:
        href1 = i.get_attribute("href")
        hrefs.append(href1)

    for m in mr_list:
        hrefm = m.get_attribute("href")
        r_hrefs.append(hrefm)
    
    # print lens for feedback
    print(len(titles))
    print(len(r_hrefs))
    
        
    time.sleep(10)
    
    # Removing the extra hrefs
    hrefs = list(dict.fromkeys(hrefs))
    print(len(hrefs))

    #fin_titles = titles[::2]
    #print(len(fin_titles)
    
    
    # I ended up prefering manually closing this one to see which products were missing reviews and/or were duplicates on amazon's side
    #driver.close()

    return hrefs, r_hrefs, titles

In [14]:
m_hrefs, mr_hrefs, m_titles = scrape(m_url)

50
46
50


In [48]:
tmp = pd.DataFrame({'title': m_titles, 'hrefs': m_hrefs})
m_df = pd.concat([tmp[~tmp['title'].isin(['Hypnotic', 'Straight No Chaser', 'Book Club: The Next Chapter'])].reset_index(drop = True), pd.DataFrame({'rhrefs': mr_hrefs})], axis = 1)
m_df.drop_duplicates(subset='title', inplace=True)

In [50]:
print(len(m_df))
m_df

38


,title,hrefs,rhrefs
0,The Super Mario Bros. Movie,https://www.amazon.com/Super-Mario-Bros-Movie/...,https://www.amazon.com/product-reviews/B0BXDGV...
1,John Wick: Chapter 4 - Bonus X-Ray Edition,https://www.amazon.com/John-Wick-Chapter-Bonus...,https://www.amazon.com/product-reviews/B0BX9MZ...
2,John Wick: Chapter 4,https://www.amazon.com/placeholder_title-John-...,https://www.amazon.com/product-reviews/B0B6QNS...
3,65,https://www.amazon.com/65-Adam-Driver/dp/B0B8T...,https://www.amazon.com/product-reviews/B0B8TVD...
4,Dungeons & Dragons: Honor Among Thieves,https://www.amazon.com/Dungeons-Dragons-Honor-...,https://www.amazon.com/product-reviews/B0B693Q...
6,Spider-Man: Into the Spider-Verse,https://www.amazon.com/Spider-Man-Into-Spider-...,https://www.amazon.com/product-reviews/B07L8Q3...
7,Creed III,https://www.amazon.com/Creed-III-Michael-B-Jor...,https://www.amazon.com/product-reviews/B0B8TFG...
8,Guy Ritchie's The Covenant,https://www.amazon.com/Guy-Ritchies-Covenant-J...,https://www.amazon.com/product-reviews/B0B8P5C...
9,Avatar: The Way of Water,https://www.amazon.com/Avatar-Way-Water-Sam-Wo...,https://www.amazon.com/product-reviews/B0B72TV...
10,Cocaine Bear,https://www.amazon.com/Cocaine-Bear-Keri-Russe...,https://www.amazon.com/product-reviews/B0B6JNB...


#### Very important, the products without reviews need to be manually removed here sadly. 

In [ ]:
m_asin = [url.split('/dp/')[1].split('/')[0] for url in m_hrefs]
mr_asin = [url.split('https://www.amazon.com/product-reviews/')[1].split('/')[0] for url in mr_hrefs]

#get the no review links
urls_not_in_mr = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] in m_asin_not_in_mr]
print(urls_not_in_mr)

In [17]:
#filter out the no-review links
m_hrefs = [url for url in m_hrefs if url.split('/dp/')[1].split('/')[0] not in m_asin_not_in_mr]
print(len(m_hrefs))

0


In [12]:
# Find and print the duplicates
names = [url.split('https://www.amazon.com/')[1].split('/')[0] for url in m_hrefs]
duplicates = [name for name, count in collections.Counter(names).items() if count > 1]
for duplicate in duplicates:
    print(f"Duplicate title: {duplicate}")
    indexes = [index for index, name in enumerate(names) if name == duplicate]
    for index in indexes:
        print(f"URL: {m_hrefs[index]}, Index: {index}")
    print()

Duplicate title: Super-Mario-Bros-Movie
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BXDGVMHS/ref=zg_bs_2958933011_sccl_1/132-2453380-0474863?psc=1, Index: 0
URL: https://www.amazon.com/Super-Mario-Bros-Movie/dp/B0BYL153QV/ref=zg_bs_2958933011_sccl_6/132-2453380-0474863?psc=1, Index: 5

Duplicate title: 65-Adam-Driver
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVD86K/ref=zg_bs_2958933011_sccl_4/132-2453380-0474863?psc=1, Index: 3
URL: https://www.amazon.com/65-Adam-Driver/dp/B0B8TVG6WW/ref=zg_bs_2958933011_sccl_37/132-2453380-0474863?psc=1, Index: 36

Duplicate title: Dungeons-Dragons-Honor-Among-Thieves
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B693QYTD/ref=zg_bs_2958933011_sccl_5/132-2453380-0474863?psc=1, Index: 4
URL: https://www.amazon.com/Dungeons-Dragons-Honor-Among-Thieves/dp/B0B6957VTZ/ref=zg_bs_2958933011_sccl_40/132-2453380-0474863?psc=1, Index: 39

Duplicate title: Spider-Man-Into-Spider-Verse-4K-UHD
URL: https://www.amazon.com/

In [25]:
m_df = pd.DataFrame({'title': m_titles, 'phref': m_refs, 'rhrefs': mr_hrefs}).drop_duplicates(subset="title")

For some reason Amazon will list the same movie twice and list movies without reviews. This led to this manual control over the dataframe after scraping. 

## Pull book URLs

In [1]:
def book_scrape(url):
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    driver.get(url)

    time.sleep(8)
# refresh to get past amazon block
    driver.refresh()

# scroll page to find all 50
    driver.execute_script("window.scrollTo(0, 10000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 14000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 15000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 16000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 17000)") 


# lists to save the data to
    hrefs = []
    r_hrefs = []
    titles = []

    # grabbing the title of each product
    title = driver.find_elements_by_css_selector("div[class^='_cDEzb_p13n-sc-css-line-clamp-']")[::2]
    for t in title:
            titles.append(t.text)
    
    # getting all elements that contain product href
    m_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@role='link']")
    
    # all elements containing 
    mr_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@title]")

    # loop through the elements to extract the href for both 
    for i in m_list:
        href1 = i.get_attribute("href")
        hrefs.append(href1)

    for m in mr_list:
        hrefm = m.get_attribute("href")
        r_hrefs.append(hrefm)
    
    # print lens for feedback
    print(len(titles))
    print(len(r_hrefs))
    
        
    time.sleep(10)
    
    # Removing the extra hrefs
    hrefs = list(dict.fromkeys(hrefs))
    print(len(hrefs))

    #fin_titles = titles[::2]
    #print(len(fin_titles)
    
    
    # I ended up prefering manually closing this one to see which products were missing reviews and/or were duplicates on amazon's side
    #driver.close()

    return hrefs, r_hrefs, titles

In [4]:
b_prod, b_review, b_title = book_scrape(b_url)

The Hill We Climb: An Inaugural Poem for the Country
Oh, the Places You'll Go!  
Happy Place
Fourth Wing (The Empyrean, 1)
Why a Daughter Needs a Dad: Celebrate Your Father Daughter Bond this Father's Day with this Special Picture Book!
Atomic Habits: An Easy & Proven Way to Build Good Habits & Break Bad Ones
The 48 Laws of Power
The Covenant of Water (Oprah's Book Club)
Lessons in Chemistry: A Novel
The Legend of Zelda™: Tears of the Kingdom – The Complete Official Guide: Collector's Edition
It Starts with Us: A Novel (2) (It Ends with Us)
Taylor Swift: A Little Golden Book Biography
Outlive: The Science and Art of Longevity
The Body Keeps the Score: Brain, Mind, and Body in the Healing of Trauma
The Housemaid
Verity
Summer Bridge Activities 1st Grade Workbooks to 2nd Grade Workbooks, Math, Reading Comprehension, Writing, Science Summer Learning Activities, 2nd Grade Workbooks All Subjects With Flash Cards
The Creative Act: A Way of Being
Summer Bridge Activities Kindergarten to 1st G

In [5]:
tmp = pd.DataFrame({'title': b_title, 'hrefs': b_prod})
b_df = pd.concat([tmp[~tmp['title'].isin(['Spider-Man: Across the Spider-Verse: The Art of the Movie','Iron Flame (The Empyrean, 2)','The Five-Star Weekend',"The Legend of Zelda™: Tears of the Kingdom – The Complete Official Guide: Collector's Edition"])].reset_index(drop = True), pd.DataFrame({'rhrefs': b_review})], axis = 1)
b_df.drop_duplicates(subset='title', inplace=True)

In [10]:
print(len(b_df))
b_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/book_df.csv')
b_df

46


,title,hrefs,rhrefs
0,The Hill We Climb: An Inaugural Poem for the C...,https://www.amazon.com/Hill-We-Climb-Inaugural...,https://www.amazon.com/product-reviews/0593465...
1,"Oh, the Places You'll Go!",https://www.amazon.com/Oh-Places-Youll-Dr-Seus...,https://www.amazon.com/product-reviews/0679805...
2,Happy Place,https://www.amazon.com/Happy-Place-Emily-Henry...,https://www.amazon.com/product-reviews/0593441...
3,"Fourth Wing (The Empyrean, 1)",https://www.amazon.com/Fourth-Wing-Empyrean-Re...,https://www.amazon.com/product-reviews/1649374...
4,Why a Daughter Needs a Dad: Celebrate Your Fat...,https://www.amazon.com/Why-Daughter-Needs-Greg...,https://www.amazon.com/product-reviews/1492667...
5,Atomic Habits: An Easy & Proven Way to Build G...,https://www.amazon.com/Atomic-Habits-Proven-Bu...,https://www.amazon.com/product-reviews/0735211...
6,The 48 Laws of Power,https://www.amazon.com/48-Laws-Power-Robert-Gr...,https://www.amazon.com/product-reviews/0140280...
7,The Covenant of Water (Oprah's Book Club),https://www.amazon.com/Covenant-Water-Abraham-...,https://www.amazon.com/product-reviews/0802162...
8,Lessons in Chemistry: A Novel,https://www.amazon.com/Lessons-Chemistry-Novel...,https://www.amazon.com/product-reviews/0385547...
9,It Starts with Us: A Novel (2) (It Ends with Us),https://www.amazon.com/Starts-Us-Novel-Ends/dp...,https://www.amazon.com/product-reviews/1668001...


## Pull Video Game URLs

In [168]:
def vg_scrape(url):
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    driver.get(url)

    time.sleep(8)
# refresh to get past amazon block
    driver.refresh()

# scroll page to find all 50
    driver.execute_script("window.scrollTo(0, 10000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 14000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 15000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 16000)") 

    time.sleep(5)

    driver.execute_script("window.scrollTo(0, 17000)") 


# lists to save the data to
    hrefs = []
    r_hrefs = []
    titles = []

    # grabbing the title of each product
    title = driver.find_elements_by_css_selector("div[class^='_cDEzb_p13n-sc-css-line-clamp-']")[::3]
    for t in title:
            titles.append(t.text)
    
    # getting all elements that contain product href
    m_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@role='link']")
    
    # all elements containing 
    mr_list = driver.find_elements_by_xpath("//a[@class='a-link-normal'][@title]")

    # loop through the elements to extract the href for both 
    for i in m_list:
        href1 = i.get_attribute("href")
        hrefs.append(href1)

    for m in mr_list:
        hrefm = m.get_attribute("href")
        r_hrefs.append(hrefm)
    
    # print lens for feedback
    print(len(titles))
    print(len(r_hrefs))
    
        
    time.sleep(10)
    
    # Removing the extra hrefs
    hrefs = list(dict.fromkeys(hrefs))
    print(len(hrefs))

    #fin_titles = titles[::2]
    #print(len(fin_titles)
    
    
    # I ended up prefering manually closing this one to see which products were missing reviews and/or were duplicates on amazon's side
    #driver.close()

    return hrefs, r_hrefs, titles

In [169]:
v_prod, v_review, v_title = vg_scrape(vg_url)

50
41
50


In [170]:
v_title

['Diablo IV - Xbox Series X',
 'Diablo IV - Standard Edition - Xbox [Digital Code]',
 'Forza Horizon 5: Standard Edition – Xbox & Windows [Digital Code]',
 'Forza Horizon 5: Premium Edition – Xbox & Windows [Digital Code]',
 'NBA 2K23 Digital Deluxe Edition - Xbox [Digital Code]',
 'Halo Infinite: Standard Edition – Xbox Series X & Xbox One',
 'Diablo IV - Ultimate Edition - Xbox [Digital Code]',
 'Star Wars Jedi: Survivor - Xbox Series X',
 'NBA 2K23 Michael Jordan Edition - Xbox [Digital Code]',
 'Saints Row Day 1 Edition - Xbox Series X',
 'Diablo IV - Digital Deluxe Edition - Xbox [Digital Code]',
 'Street Fighter 6 - XBX',
 'Street Fighter 6 - PRE-PURCHASE Standard - Xbox Series X|S [Digital Code]',
 'Dead Space - Xbox Series X',
 "ASSASSIN'S CREED MIRAGE - DELUXE EDITION, XBOX X",
 'Elden Ring - Xbox Series X',
 'Street Fighter 6 Deluxe Edition - XBX',
 'Star Wars Jedi: Survivor Standard - Xbox Series X|S [Digital Code]',
 'AEW: Fight Forever - Xbox One/ Xbox Series X',
 'Residen

In [173]:
tmp = pd.DataFrame({'title': v_title, 'hrefs': v_prod})
v_df = pd.concat([tmp[~tmp['title'].isin(['Aliens: Dark Descent XSX|XB1','We Love Katamari REROLL + Royal Reverie - Xbox Series X|Xbox One','Sonic Origins Plus - Xbox Series X','Mortal Kombat 1 Premium Edition - Xbox Series X', 'AEW: Fight Forever - Xbox One/ Xbox Series X',"ASSASSIN'S CREED MIRAGE - DELUXE EDITION, XBOX X", 'Street Fighter 6 - PRE-PURCHASE Standard - Xbox Series X|S [Digital Code]','Diablo IV - Digital Deluxe Edition - Xbox [Digital Code]','Diablo IV - Xbox Series X'])].reset_index(drop = True), pd.DataFrame({'rhrefs': v_review})], axis = 1)
v_df.drop_duplicates(subset='title', inplace=True)

In [180]:
# some of the video games have a review link but no reviews
vg_df = v_df[~v_df['title'].isin(['Diablo IV - Standard Edition - Xbox [Digital Code]','Street Fighter 6 - XBX'])].reset_index(drop = True)
vg_df

,title,hrefs,rhrefs
0,Forza Horizon 5: Standard Edition – Xbox & Win...,https://www.amazon.com/Forza-Horizon-Standard-...,https://www.amazon.com/product-reviews/B09DCTN...
1,Forza Horizon 5: Premium Edition – Xbox & Wind...,https://www.amazon.com/Forza-Horizon-Premium-W...,https://www.amazon.com/product-reviews/B09DCR6...
2,NBA 2K23 Digital Deluxe Edition - Xbox [Digita...,https://www.amazon.com/NBA-2K23-Digital-Deluxe...,https://www.amazon.com/product-reviews/B0BGZG7...
3,Halo Infinite: Standard Edition – Xbox Series ...,https://www.amazon.com/Halo-Infinite-Xbox-One-...,https://www.amazon.com/product-reviews/B07SL6Y...
4,Diablo IV - Ultimate Edition - Xbox [Digital C...,https://www.amazon.com/Diablo-IV-Ultimate-Xbox...,https://www.amazon.com/product-reviews/B0BZTPL...
5,Star Wars Jedi: Survivor - Xbox Series X,https://www.amazon.com/Star-Wars-Jedi-Survivor...,https://www.amazon.com/product-reviews/B0BP5L9...
6,NBA 2K23 Michael Jordan Edition - Xbox [Digita...,https://www.amazon.com/NBA-2K23-Michael-Jordan...,https://www.amazon.com/product-reviews/B0BGZHV...
7,Saints Row Day 1 Edition - Xbox Series X,https://www.amazon.com/Saints-Row-2022-Xbox-X/...,https://www.amazon.com/product-reviews/B09DN32...
8,Dead Space - Xbox Series X,https://www.amazon.com/Dead-Space-Xbox-X/dp/B0...,https://www.amazon.com/product-reviews/B0BGW86...
9,Elden Ring - Xbox Series X,https://www.amazon.com/Elden-Ring-Xbox-One/dp/...,https://www.amazon.com/product-reviews/B07SMBN...


In [181]:
vg_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/vg_urls.csv')

## Scrape Movies' reviews and details

In [105]:
def scrape_movie_details(list_urls):

    # lists for movie details
    title = []
    imdb = []
    amazon_rating = []
    review_count = []
    length = []
    year = []
    rating = []
    price = []


    # activate driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()

        title.append(driver.find_element_by_xpath("//h1[@data-automation-id='title']").text)
        # some products that are new do not have official ratings and a bypass needs to be created. Used the NoSuchElement exception from selenium to bypass this

        # imbd rating
        try:
            im = driver.find_element_by_xpath("//span[@data-automation-id='imdb-rating-badge']").text
            imdb.append(im.split(" ")[1])
        except NoSuchElementException:
            print("No IMDB Rating")
            continue

        
        # amazon rating
        try:
            ar = driver.find_element_by_xpath("//span[@class='_3mK_sl _3NWEVD']").get_attribute("aria-label")
            amazon_rating.append(ar.split(" ")[1])
        except NoSuchElementException:
            print("No Amazon Rating")
            amazon_rating.append(np.nan)
        
        # amount of amazon reviews
        try:
            review_count.append(driver.find_element_by_xpath("//span[@class='_1Y_bt3']").text)
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        # movie runtime
        try:  
            length.append(driver.find_element_by_xpath("//span[@data-automation-id='runtime-badge']").text)
        except NoSuchElementException:
            print("No Amazon Runtime")
            length.append(np.nan)
        
        # year released
        year.append(driver.find_element_by_xpath("//span[@data-automation-id='release-year-badge']").text)

        # age rating G|PG etc..
        rating.append(driver.find_element_by_xpath("//span[@class='G8xF_x']").text)
        
        # price in whole dollar
        try:
            pr = driver.find_element_by_xpath("//button[@class='_2X_Irl tvod-button av-button']").text
            price.append(re.findall("\d+", pr)[0])
        except NoSuchElementException:
            print("No Listed Price")
            price.append(np.nan)

        time.sleep(random.randint(8,15))
    
    # close driver
    driver.close()   

    # create dataframe
    data = pd.DataFrame({"title": title, "imbd_rating": imdb,"amazon_rating":amazon_rating, "review_count": review_count, "runtime": length, 
                         "year": year, "rating": rating, "price": price})
    
    return data


        

def scrape_reviews(list_urls):

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        # lists for reviews
        rating = []
        review = [] 
        name = []
        title = []
        date_place = [] 
        movie = []

        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 800):

            # using the url plus this handy snippet to loop through all review pages
            driver.get(i+f"?ie=UTF8&pageNumber={n}")
            
            time.sleep(random.randint(5,12))
            driver.refresh()

            # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                if a.text == '':
                    continue
                else:
                    rev_check.append(a.text)

            # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:

                track_n = 0
                track_r = 0
                track_rat = 0
                track_t = 0
                track_dp = 0
                
                # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                # rating
                ratings = driver.find_elements_by_xpath("//i[@class][@data-hook]//span")[3:]

                # review title
                rev_title = driver.find_elements_by_xpath("//*[@data-hook='review-title'][@class]/span")

                # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                # movie name
                movies = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)
                    track_r += 1

                for na in names:
                    if na.text == '':
                        continue
                    else:
                        name.append(na.text)
                        track_n += 1

                for rat in ratings:
                    #if rat.text != '':
                        #if rat.text.split()[1] == 'out':
                    rating.append(rat.get_attribute('innerHTML').split()[0])
                    track_rat += 1
                    

                            
                for t in rev_title:
                    if t.text != '':
                        title.append(t.text)
                        track_t += 1
                            
                for d in dateplaces:
                    date_place.append(d.text)
                    track_dp += 1
                    
                for m in movies:
                    for mo in [m.text] * len(rev_check):
                        movie.append(mo)

                # give amazon a little break     
                time.sleep(random.randint(8,15))

                tracker = [track_n, track_t, track_rat, track_r, track_dp]
                if all(item == tracker[0] for item in tracker):
                    print(n, "This page is good")
                else:
                    print(n, "This page is broken")
                    break

            # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    



        # see lens to find out if there were any mistakes
        print("name len:", len(name))
        print("review len:", len(review))
        print("rating len:", len(rating))
        print("title len:", len(title))
        print("dateplace len:", len(date_place))
        print("movie len:", len(movie))

        savefile = str(str(movie[0]).replace(" ", "").replace(":", "").replace(".", "") + "_review_df")

        print(name)
        # create review dataframe
        data = pd.DataFrame({"name": name, "movie_title": movie, "review_title": title, "review": review, "rating": rating, "date_place": date_place})
        data.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/movies/{}.csv'.format(savefile))

    # close driver
    driver.close()

In [54]:
movie_details_df = scrape_movie_details(m_df['hrefs'])

No Amazon Runtime


In [55]:
movie_details_df.sort_values('title', ascending=False)          

,title,imbd_rating,amazon_rating,review_count,runtime,year,rating,price
26,Vanderpump Rules,5.8,4.7,266,NaN,2023,TV-14,17
17,The Whale,7.7,4.3,"11,019",1 h 56 min,2022,18+,5
0,The Super Mario Bros. Movie,7.2,4.8,"1,862",1 h 32 min,2023,PG,24
36,The Ritual Killer,4.5,3.7,"2,331",1 h 31 min,2023,16+,4
19,The Pope's Exorcist,6.1,4.3,986,1 h 43 min,2023,R,19
24,The Fast and the Furious,6.8,4.7,"1,215",1 h 47 min,2001,PG-13,3
33,The Croods: A New Age,6.9,4.8,"59,792",1 h 35 min,2020,PG,3
5,Spider-Man: Into the Spider-Verse,8.4,4.8,"57,097",1 h 56 min,2018,PG,4
12,Sisu,7.0,4.5,229,1 h 31 min,2023,R,19
16,Shazam! Fury of the Gods,6.0,4.3,240,2 h 10 min,2023,PG-13,5


In [56]:
movie_details_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/md_df.csv')

In [107]:
scrape_reviews(m_df['rhrefs'])

5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
1 This page is good
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
2 This page is good
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
3 This page is good
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
3.0 out of 5 stars
4 This page is good
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
5.0 out of 5 stars
4.0 out of 5 stars
5.0 out of 5 stars
5.0 out 

In [65]:
m_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/mref.csv')

## Scraping Books

In [147]:
def scrape_book_details(list_urls):
    book = []
    amazon_rating = []
    review_count = []
    year = []
    price = []
    author = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        book.append(driver.find_element_by_xpath("//span[@id='productTitle']").text)
        
        ar = driver.find_elements_by_xpath("//span[@class='a-size-base a-color-base']")
        
        tmpar = []
        for a in ar:
            if a.text != '':
                if a.text.split()[0] not in set(['Get', '&', 'Total']):
                    tmpar.append(a.text)
        amazon_rating.append(tmpar[0])
        book_author = driver.find_elements_by_xpath("//span[@class='author notFaded']")[0]
        author.append(book_author.text.replace(",", "").replace("(Author)","").replace("(Compiler)", ""))
        #for b in book_author:
            #if b.text.split()[-1].replace(",","") == '(Author)':
               # author.append(b.text.replace("(Author)", "").replace(",",""))
            #elif b.text.split()[-1].replace(",","") == '(Author)'
        
        
        try:
            rc = driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").text
            review_count.append(rc.split()[0].replace(",",""))
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        year.append(driver.find_element_by_xpath("//span[@id='productSubtitle']").text.split()[-1])
        

        price.append(driver.find_element_by_xpath("//span[@class='a-size-base a-color-price a-color-price']").text)

        time.sleep(random.randint(8,15))
    
    driver.close()   
    print("book len", len(book))
    print("author len", len(author))
    print("amazon rating", len(amazon_rating))
    print("review count", len(review_count))
    print("year", len(year))
    print("price", len(price))
    data = pd.DataFrame({"title": book, "author": author, "amazon_rating":amazon_rating, "review_count": review_count,
                         "year": year, "price": price})
    return data

    

In [114]:
def scrape_book_reviews(list_urls):

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        # lists for reviews
        rating = []
        review = [] 
        name = []
        title = []
        date_place = [] 
        book = []

        
        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 800):

                # using the url plus this handy snippet to loop through all review pages
            try:
                driver.get(i+f"?ie=UTF8&pageNumber={n}")
                # The Amazon pages sometimes require a refresh to access the page
                time.sleep(random.randint(5,10))
                driver.refresh()
            except TimeoutException:
                # For these longer scrapes, there has been random timeouts that just require the page to be refreshed a 2nd time to fix
                time.sleep(random.randint(5,10))
                driver.refresh()


                # grabbing names past first two that are always there to decide if page has reviews
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]
            rev_check = []
            for a in names:
                if a.text == '':
                    continue
                else:
                    rev_check.append(a.text)

                # if page has 1  or more name (reviews) scrape info below
            if len(rev_check) > 0:

                track_n = 0
                track_r = 0
                track_rat = 0
                track_t = 0
                track_dp = 0
                    
                    # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                    # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                    # rating
                ratings = driver.find_elements_by_xpath("//i[@class][@data-hook]//span")[3:]

                    # review title
                rev_title = driver.find_elements_by_xpath("//*[@data-hook='review-title'][@class]/span")

                    # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                    # book name
                books = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                    # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)
                    track_r += 1

                for na in names:
                    if na.text == '':
                        continue
                    else:
                        name.append(na.text)
                        track_n += 1

                for rat in ratings:
                    rating.append(rat.get_attribute('innerHTML').split()[0])
                    track_rat += 1
                        

                                
                for t in rev_title:
                    if t.text != '':
                        title.append(t.text)
                        track_t += 1
                                
                for d in dateplaces:
                    date_place.append(d.text)
                    track_dp += 1
                        
                for m in books:
                    for mo in [m.text] * len(rev_check):
                        book.append(mo)

                    # give amazon a little break     
                time.sleep(random.randint(8,12))

                tracker = [track_n, track_t, track_rat, track_r, track_dp]
                if all(item == tracker[0] for item in tracker):
                    print(n, "This page is good")
                else:
                    print(n, "This page is broken")
                    break

                # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_check) == 0:
                print("There are no more reviews for this product!")
                break
    



        # see lens to find out if there were any mistakes
        print("name len:", len(name))
        print("review len:", len(review))
        print("rating len:", len(rating))
        print("title len:", len(title))
        print("dateplace len:", len(date_place))
        print("book len:", len(book))

        savefile = str(str(book[0]).replace(",", "").replace(" ", "").replace(":", "").replace(".", "").replace("?", "") + "_review_df")

        # create review dataframe
        data = pd.DataFrame({"name": name, "movie_title": book, "review_title": title, "review": review, "rating": rating, "date_place": date_place})
        data.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/books/{}.csv'.format(savefile))

    # close driver
    driver.close()

In [148]:
book_details = scrape_book_details(b_df['hrefs'])

book len 46
author len 46
amazon rating 46
review count 46
year 46
price 46


In [149]:
# 48 laws of power, 
book_details

,title,author,amazon_rating,review_count,year,price
0,The Hill We Climb: An Inaugural Poem for the C...,Amanda Gorman,4.9,16043,2021,$9.98
1,"Oh, the Places You'll Go!",Dr. Seuss,4.9,46583,1990,$8.48
2,Happy Place,Emily Henry,4.4,20583,2023,$15.49
3,"Fourth Wing (The Empyrean, 1)",Rebecca Yarros,4.9,11488,2023,$17.99
4,Why a Daughter Needs a Dad: Celebrate Your Fat...,Gregory E. Lang,4.9,14538,2019,$7.19
5,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,4.8,110526,2018,$14.89
6,The 48 Laws of Power,Robert Greene,4.7,64755,2000,$13.86
7,The Covenant of Water (Oprah's Book Club),Abraham Verghese,4.7,3230,2023,$19.58
8,Lessons in Chemistry: A Novel,Bonnie Garmus,4.6,164519,2022,$17.99
9,It Starts with Us: A Novel (2) (It Ends with Us),Colleen Hoover,4.5,116065,2022,$10.49


In [150]:
book_details.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/book_details.csv')

In [ ]:
scrape_book_reviews(b_df['rhrefs'])

## Scraping Video Games

In [243]:
def scrape_games_details(list_urls):
    game = []
    amazon_rating = []
    review_count = []
    rating = []
    price = []
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')
    for i in list_urls:
        driver.get(i)
        
        time.sleep(random.randint(8,15))
        driver.refresh()
        game.append(driver.find_element_by_xpath("//span[@id='productTitle']").text)

        # some products that are new do not have official ratings and a bypass needs to be created
        
        ar = driver.find_element_by_xpath("//span[@data-hook='rating-out-of-text']")
        amazon_rating.append(ar.text.split()[0])
        print(ar.text)
        
           
        
        try:
            review_count.append(driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").text.split()[0])
        except NoSuchElementException:
            print("No Amazon Reviews")
            review_count.append(np.nan)

        try:
            price.append(driver.find_element_by_xpath("//span[@class='a-price-whole']").text.replace('.','')+'.'+driver.find_element_by_xpath("//span[@class='a-price-fraction']").text)
        except NoSuchElementException:
            print("Product Not Available")
            price.append(np.nan)

        time.sleep(random.randint(8,15))

    print("game", len(game))
    print("amazon rating", len(amazon_rating))
    print("review_count", len(review_count))
    print("price", len(price))

    
    
    driver.close()  

    data = pd.DataFrame({"title": game,"amazon_rating":amazon_rating, 
                         "review_count": review_count,
                         "price": price})
    
    return data

In [307]:
def scrape_vg_reviews(list_urls):

    # open driver
    driver = webdriver.Edge(executable_path='C:\Program Files\Microsoft\msedgedriver.exe')

    # loop through urls
    for i in list_urls:
        # lists for reviews
        rating = []
        review = [] 
        name = []
        title = []
        date_place = [] 
        game = []

        driver.get(i+f"?ie=UTF8&pageNumber=1")
        time.sleep(random.randint(5,10))
        driver.refresh()
        
        size = driver.find_elements_by_xpath("//span[@class='a-profile-name']")
        sizes = []
        for si in size:
            if si != '':
                sizes.append(si.text)


        
        # loop through range of pages | stopped at 900(or 9,000 reviews) to save time and memory
        for n in range(1, 800):

                # using the url plus this handy snippet to loop through all review pages
            try:
                driver.get(i+f"?ie=UTF8&pageNumber={n}")
                # The Amazon pages sometimes require a refresh to access the page
                time.sleep(random.randint(5,10))
                driver.refresh()
            except TimeoutException:
                # For these longer scrapes, there has been random timeouts that just require the page to be refreshed a 2nd time to fix
                time.sleep(random.randint(5,10))
                driver.refresh()


                # grabbing names past first two that are always there to decide if page has reviews
            revs = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")
            rev_num = []
            for rev in revs:
                rev_num.append(rev.text)
            names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")
            rev_check = []
            for a in names:
                if a.text == '':
                    continue
                else:
                    rev_check.append(a.text)

                # if page has 1  or more name (reviews) scrape info below
            if len(sizes) > 10 and len(rev_num) > 0:

                track_n = 0
                track_r = 0
                track_rat = 0
                track_t = 0
                track_dp = 0
                    
                    # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")[2:]

                    # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                    # rating
                ratings = driver.find_elements_by_xpath("//i[@class][@data-hook]//span")[3:]

                    # review title
                rev_title = driver.find_elements_by_xpath("//*[@data-hook='review-title'][@class]/span")

                    # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                    # book name
                games = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                    # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)
                    track_r += 1

                for na in names:
                    if na.text == '':
                        continue
                    else:
                        name.append(na.text)
                        track_n += 1

                for rat in ratings:
                    rating.append(rat.get_attribute('innerHTML').split()[0])
                    track_rat += 1
                        

                                
                for t in rev_title:
                    if t.text != '':
                        title.append(t.text)
                        track_t += 1
                                
                for d in dateplaces:
                    date_place.append(d.text)
                    track_dp += 1
                        
                for m in games:
                    for mo in [m.text] * len(rev_num):
                        game.append(mo)

                    # give amazon a little break     
                time.sleep(random.randint(8,12))

                tracker = [track_n, track_t, track_rat, track_r, track_dp]
                if all(item == tracker[0] for item in tracker):
                    print(n, "This page is good")
                else:
                    print(n, "This page is broken")
                    break

    
            
            elif len(sizes) < 11 and len(rev_num) > 0:

                track_n = 0
                track_r = 0
                track_rat = 0
                track_t = 0
                track_dp = 0
                    
                    # user name
                names = driver.find_elements_by_xpath("//span[@class='a-profile-name']")

                    # review body
                reviews = driver.find_elements_by_xpath("//span[@class='a-size-base review-text review-text-content']")

                    # rating
                ratings = driver.find_elements_by_xpath("//i[@class][@data-hook]//span")[1:]

                    # review title
                rev_title = driver.find_elements_by_xpath("//*[@data-hook='review-title'][@class]/span")

                    # date and place
                dateplaces = driver.find_elements_by_xpath("//span[@data-hook='review-date']")

                    # book name
                games = driver.find_elements_by_xpath("//a[@data-hook='product-link']")

                    # loop through to extract the text for each review / element
                for rev in reviews:
                    review.append(rev.text)
                    track_r += 1

                for na in names:
                    if na.text == '':
                        continue
                    else:
                        name.append(na.text)
                        track_n += 1

                for rat in ratings:
                    rating.append(rat.get_attribute('innerHTML').split()[0])
                    track_rat += 1
                        

                                
                for t in rev_title:
                    if t.text != '':
                        title.append(t.text)
                        track_t += 1
                                
                for d in dateplaces:
                    date_place.append(d.text)
                    track_dp += 1
                        
                for m in games:
                    for mo in [m.text] * len(rev_check):
                        game.append(mo)

                    # give amazon a little break     
                time.sleep(random.randint(8,12))

                tracker = [track_n, track_t, track_rat, track_r, track_dp]
                if all(item == tracker[0] for item in tracker):
                    print(n, "This page is good")
                else:
                    print(n, "This page is broken")
                    break
                # if the amount of names is equal to 0, break to next movie as there are no more reviews
            elif len(rev_num) == 0:
                print("There are no more reviews for this product!")
                break
    



        # see lens to find out if there were any mistakes
        print("name len:", len(name))
        print("review len:", len(review))
        print("rating len:", len(rating))
        print("title len:", len(title))
        print("dateplace len:", len(date_place))
        print("game len:", len(game))

        savefile = str(str(game[0]).replace("/", "").replace("'","").replace("|","").replace(",", "").replace(" ", "").replace(":", "").replace(".", "").replace("?", "").replace(")","").replace("(", "").replace("]","").replace("[", "") + "_review_df")

        # create review dataframe
        data = pd.DataFrame({"name": name, "game_title": game, "review_title": title, "review": review, "rating": rating, "date_place": date_place})
        data.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/games/{}.csv'.format(savefile))

    # close driver
    driver.close()

In [231]:
len(vg_df)

39

In [ ]:
vgd_df = scrape_games_details(vg_df['hrefs'])

In [248]:
vgd_df.to_csv(r'C:/Users/whitm/OneDrive/Desktop/ADS509/game_details.csv')
vgd_df

,title,amazon_rating,review_count,price
0,Forza Horizon 5: Standard Edition – Xbox & Win...,4.6,645,28.99
1,Forza Horizon 5: Premium Edition – Xbox & Wind...,4.6,467,44.99
2,NBA 2K23 Digital Deluxe Edition - Xbox [Digita...,3.8,43,12.00
3,Halo Infinite: Standard Edition – Xbox Series ...,4.7,"4,103",149.99
4,Diablo IV - Ultimate Edition - Xbox [Digital C...,3,4,99.99
5,Star Wars Jedi: Survivor - Xbox Series X,4.4,78,69.88
6,NBA 2K23 Michael Jordan Edition - Xbox [Digita...,4.2,40,20.00
7,Saints Row Day 1 Edition - Xbox Series X,4.3,359,19.99
8,Dead Space - Xbox Series X,4.7,206,44.99
9,Elden Ring - Xbox Series X,4.6,"1,575",45.49


In [ ]:
scrape_vg_reviews(vg_df['rhrefs'])